Using sbpy.data.Names
===========

[sbpy.data.Names](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Names.html#sbpy.data.Names) provides functionality to parse asteroid and comet names, numbers, and designations and to identify an object's nature (asteroid or comet) based on the provided identifier.

Parsing asteroid and comet names
--------------------------------

Imagine you get data from somebody else, involving a somewhat random list of targets like the following two:

In [1]:
asteroids = ['3200 Phaethon', '2018 AA', '(433)', '1036 Ganymed (1924 TD)', 'K14J00D']
comets = ['2P/Encke', '259P', '73P/Schwassmann Wachmann 3 C', 'X/1106 C1', 'P/1994 N2 (McNaught-Hartley)']

Dealing with a target list like this might be complicated, since different formats are involved. For instance, the list cannot easily matched with another target list, as it is not straightforward to know whether target names, numbers, or designations are used. [sbpy.data.Names.parse_asteroid](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Names.html#sbpy.data.Names.parse_asteroid) and [sbpy.data.Names.parse_comet](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Names.html#sbpy.data.Names.parse_comet) can be used to classify and disentangle these identifiers:

In [2]:
from sbpy.data import Names

for asteroid in asteroids:
    print(asteroid, '->', Names.parse_asteroid(asteroid))
    
for comet in comets:
    print(comet, '->', Names.parse_comet(comet))
    

3200 Phaethon -> {'number': 3200, 'name': 'Phaethon'}
2018 AA -> {'desig': '2018 AA'}
(433) -> {'number': 433}
1036 Ganymed (1924 TD) -> {'number': 1036, 'name': 'Ganymed', 'desig': '1924 TD'}
K14J00D -> {'desig': '2014 JD'}
2P/Encke -> {'type': 'P', 'number': 2, 'name': 'Encke'}
259P -> {'type': 'P', 'number': 259}
73P/Schwassmann Wachmann 3 C -> {'type': 'P', 'number': 73, 'name': 'Schwassmann Wachmann 3 C'}
X/1106 C1 -> {'type': 'X', 'desig': '1106 C1'}
P/1994 N2 (McNaught-Hartley) -> {'type': 'P', 'desig': '1994 N2', 'name': 'McNaught-Hartley'}


Each function dissects the identifier string provided and tries to find patterns that match those of names, numbers, and designations using regular expressions. The output of each function is a dictionary with the found patterns. Note that packed designations and numbers are un-packed. 

If [sbpy.data.Names.parse_asteroid](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Names.html#sbpy.data.Names.parse_asteroid) is applied to a comet identifier (and vice versa), a `TargetNameParseError` is raised:

In [3]:
Names.parse_asteroid(comets[0])

TargetNameParseError: 2P/Encke does not appear to be an asteroid identifier

Distinguishing between asteroid and comet identifiers
--------------------------------------------

[sbpy.data.Names.asteroid_or_comet](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Names.html#sbpy.data.Names.asteroid_or_comet) can be used to identify asteroid and comet identifiers:

In [4]:
for obj in asteroids + comets:
    print(obj, '->', Names.asteroid_or_comet(obj))

3200 Phaethon -> asteroid
2018 AA -> asteroid
(433) -> asteroid
1036 Ganymed (1924 TD) -> asteroid
K14J00D -> asteroid
2P/Encke -> comet
259P -> comet
73P/Schwassmann Wachmann 3 C -> comet
X/1106 C1 -> comet
P/1994 N2 (McNaught-Hartley) -> comet


If the object identifier is ambiguous or does not match a pattern associated with asteroid or comet identifiers, `None` is returned:

In [5]:
print(Names.asteroid_or_comet('Spam Spam Eggs and Spam'))

None


Limitations of the methods
------------

Keep in mind that neither of the methods in [sbpy.data.Names](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Names.html) knows anything about the objects associated with the object identifier; there is no database query being performed. All information the methods have available comes from the provided identifier string. 

This lack of information can lead to some ambiguity.

Consider the following case in which the identifier string only consists of a name:

In [6]:
obj = "Ceres"

print(Names.parse_asteroid(obj), Names.parse_comet(obj))

{'name': 'Ceres'} {'name': 'Ceres'}


Both [sbpy.data.Names.parse_asteroid](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Names.html#sbpy.data.Names.parse_asteroid) and [sbpy.data.Names.parse_comet](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Names.html#sbpy.data.Names.parse_comet) extract a target name in this case - there is no `TargetNameParseError` raised. Hence, the target type identification is ambiguous:

In [7]:
print(Names.asteroid_or_comet(obj))

None


This behavior can be easily explained: both asteroids and comets have names and without additional information, the pattern matching itself is not able to make definitive decisions on the target nature. 

More complicated is the situation for comets that have asteroid designations (since they were originally classified as asteroids). If only a designation is passed to `asteroid_or_comet`, the object is classified as an asteroid as this is the most likely nature of the target:

In [8]:
obj = "2014 OE4"
print(Names.asteroid_or_comet(obj))

asteroid


In order to unambiguously define the object as a comet, you should add the comet type to the designation:

In [9]:
obj = "C/2014 OE4"
print(Names.asteroid_or_comet(obj))

comet
